In [20]:
from agents import Agent, trace, Runner, function_tool
from dotenv import load_dotenv
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

## agent Workflow



In [3]:
instruction1 = " You are asales agent working for ComplAI, \
    A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai. \
        you wirte a professional, serious cold email"

In [4]:
instruction1

' You are asales agent working for ComplAI,     A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai.         you wirte a professional, serious cold email'

In [ ]:
sales_agent = Agent(
    name = "Professional sales Agent",
    instructions = instruction1,
    model = "gpt-4o-mini"
)

In [6]:
sales_agent


Agent(name='Professional sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=' You are asales agent working for ComplAI,     A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai.         you wirte a professional, serious cold email', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [13]:
result = Runner.run_streamed(sales_agent, "write a cold sales email ") # starts the agent , this run_Stremed tells agent to write the cold sales

async for event in result.stream_events(): # get events one by one gives a small poevces of ouput instesas od waole email
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):  # pick only the text poeces 
        print(event.data.delta, end = '', flush=True) # print each chunk immediately , so the email appears words ny words 


# is just a way to stream the AI’s response live instead of waiting for the full answer at the end.

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well.

As your organization continues to navigate the complexities of data security and compliance, I wanted to introduce you to ComplAI—a powerful SaaS tool designed specifically to simplify SOC 2 compliance and prepare your team for audits.

Our AI-driven platform offers:

- **Automated Documentation**: Generate and maintain necessary documentation effortlessly.
- **Real-Time Monitoring**: Stay compliant with continuous monitoring and alerts.
- **Seamless Integration**: Connect with your existing tools for a streamlined experience.
- **Expert Guidance**: Access resources and expertise to guide you through audits.

With ComplAI, you can reduce the time and effort spent on compliance tasks, allowing your team to focus on what truly matters—growing your business.

I’d love to schedule a brief call to discuss how ComplAI can support your compliance goals. Are you available for a 

In [10]:
result


RunResultStreaming(input='write a cold sales email ', new_items=[], raw_responses=[], final_output=None, input_guardrail_results=[], output_guardrail_results=[], context_wrapper=RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0)), current_agent=Agent(name='Professional sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions=' You are asales agent working for ComplAI,     A COMPANY THAT prvoides a saas tool for ensuring soc2 compliance and preparing for audits. powered by ai.         you wirte a professional, serious cold email', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbo

In [14]:
sales_picker = Agent(
    name = " select the best sales cold email",
    instructions = " You pick the best cold sales email from the given options. \
    Imagine you are a customer and pick the one you are most likely to respond to. \
    Do not give an explanation; reply with the selected email only.",
    model = "gpt-4o-mini"
)


In [15]:

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [22]:
sales_agent1 = Agent(
    name = "Professional sales Agent",
    instructions = instruction1,
    model = "gpt-4o-mini"
)
sales_agent2 = Agent(
    name = "Professional sales Agent",
    instructions = instructions2,
    model = "gpt-4o-mini"
)
sales_agent3 = Agent(
    name = "Professional sales Agent",
    instructions = instructions3,
    model = "gpt-4o-mini"
)

In [23]:
message = "Write a cold sales email"

with trace("pick the best email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )


outputs = [result.final_output for result in results]

In [25]:
 emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

In [27]:
best = await Runner.run(sales_picker, emails)

In [28]:
best

RunResult(input="Cold sales emails:\n\nSubject: Simplify Your SOC 2 Compliance with ComplAI\n\nHi [Recipient's Name],\n\nI hope this message finds you well.\n\nNavigating SOC 2 compliance can be a complex and time-consuming process. At ComplAI, we understand the challenges companies face when preparing for audits and ensuring compliance. That’s why we've developed an AI-powered SaaS tool designed to streamline your compliance efforts and enhance your audit readiness.\n\n**Key Benefits of Using ComplAI:**\n\n- **Automated Documentation:** Our platform simplifies the documentation process, providing templates and guidance to help you stay compliant with minimal effort.\n- **Real-Time Monitoring:** Keep track of your compliance status effortlessly, with real-time updates and alerts to ensure you're always prepared for your next audit.\n- **Expert Insights:** Gain access to our library of resources and expert advice tailored to your industry's specific compliance requirements.\n\nMany orga

In [29]:
best.final_output

"Subject: Your SOC2 Compliance Journey Just Got a Makeover! ✨\n\nHi [Recipient's Name],\n\nI hope this email finds you wading through the paperwork with a smile (or a caffeinated beverage, at least)! ☕ \n\nI’m reaching out from ComplAI, where we believe that getting SOC2 compliant shouldn’t feel like climbing Everest—more like a stroll through a park, complete with a barbecue. 🌳🔥 \n\nOur AI-powered tool does the heavy lifting, handling everything from documentation to audit prep. Think of it as your trusty sidekick, but instead of a cape, it comes with compliance magic! 🦸\u200d♂️✨\n\nImagine being able to check “SOC2 compliance” off your to-do list without breaking a sweat. Sounds dreamy, right? 😴💭 \n\nLet’s schedule a quick call! I promise, no legalese or jargon—just some good ol’ common sense and a sprinkle of humor. \n\nLooking forward to hearing from you!\n\nBest,  \n[Your Name]  \nComplAI – Making Compliance Less of a Compliance-tion!  \n[Your Contact Information]  \n[Your LinkedI

### tools and agent interaction

Wrap the function with the decorator @`function_too`


In [ ]:
def send_email(body:str):
    send_grid = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))